In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from pynq.overlays.h264 import H264Overlay
from pynq.lib.video import *

base = H264Overlay("h264.bit")
hdmi_in = base.video.hdmi_in

load ready


In [4]:
from pynq import MMIO
rgb2yuv = MMIO(base.ip_dict['h264/rgb2yuv_with_axi_0']['phys_addr'], 0x10000)
h264 = MMIO(base.ip_dict['h264/h264enc_with_axi_0']['phys_addr'], 0x10000)

from h264py.h264 import H264
h264_send = H264()

from pynq import Xlnk
xlnk = Xlnk()

In [5]:
size = 1920*1088*4

## repeat from here

In [10]:
with hdmi_in.configure(pixelformat=PIXEL_RGBA):
    hdmi_in.start()
    cma_recv = xlnk.cma_array((size,), dtype=np.uint8)
    result = xlnk.cma_array((size,), dtype=np.uint8)
    start = time.time()
    for i in range(2000):
        
        cma_send = hdmi_in.readframe()
        
        rgb2yuv.write(0x04, cma_send.physical_address)
        rgb2yuv.write(0x08, cma_recv.physical_address)
        rgb2yuv.write(0x0c, 1088)
        rgb2yuv.write(0x10, 1920)
        rgb2yuv.write(0x14, 1920*1088)
        rgb2yuv.write(0x00, 1)
        rgb2yuv.write(0x00, 0)
        while rgb2yuv.read(0x18)==1: pass
        while rgb2yuv.read(0x18)==0: pass
        
        h264.write(0x04, 0x00004377) # length 1088*1920
        h264.write(0x08, 0)
        h264.write(0x14, cma_recv.physical_address)
        h264.write(0x18, result.physical_address)
        h264.write(0x00, 1)
        while h264.read(0x24)==1: pass
        while h264.read(0x24)==0: pass

        h264_size = h264.read(0x1c)
        h264_send.send_frame(result[:h264_size])

        cma_send.freebuffer()
        
    print(time.time()-start)

cma_recv.freebuffer()
result.freebuffer()

196.29687213897705


In [ ]:
xlnk.xlnk_reset()